In [1]:
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import warnings
import numpy as np
warnings.filterwarnings('ignore')
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_categorical_dtype
import janitor

In [2]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

In [3]:
train

,pregnancy_occurence,glucose_concentration,blood_pressure,triceps_foldness,serum_insulin,bmi,predigree_function,age,target
0,2,84,0,0,0,0.0,0.304,21,0
1,9,112,82,24,0,28.2,1.282,50,1
2,1,139,46,19,83,28.7,0.654,22,0
3,0,161,50,0,0,21.9,0.254,65,0
4,6,134,80,37,370,46.2,0.238,46,1
...,...,...,...,...,...,...,...,...,...
609,5,139,64,35,140,28.6,0.411,26,0
610,1,96,122,0,0,22.4,0.207,27,0
611,10,101,86,37,0,45.6,1.136,38,1
612,0,141,0,0,0,42.4,0.205,29,1


In [4]:
#train = train.drop(["cabin", "name", "passengerid"], axis=1)
#test = test.drop(["cabin", "name", "passengerid"], axis = 1)

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   pregnancy_occurence    614 non-null    int64  
 1   glucose_concentration  614 non-null    int64  
 2   blood_pressure         614 non-null    int64  
 3   triceps_foldness       614 non-null    int64  
 4   serum_insulin          614 non-null    int64  
 5   bmi                    614 non-null    float64
 6   predigree_function     614 non-null    float64
 7   age                    614 non-null    int64  
 8   target                 614 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 43.3 KB


In [6]:
cat_col = []
num_col = []
for col in list(train.columns):
    if is_categorical_dtype(train[col]):
        cat_col.append(col)
    elif is_numeric_dtype(train[col]):
        num_col.append(col)

In [7]:
num_col, len(num_col)

(['pregnancy_occurence',
  'glucose_concentration',
  'blood_pressure',
  'triceps_foldness',
  'serum_insulin',
  'bmi',
  'predigree_function',
  'age',
  'target'],
 9)

In [8]:
num_col.remove("target")

In [9]:
cat_col

[]

In [10]:
train

,pregnancy_occurence,glucose_concentration,blood_pressure,triceps_foldness,serum_insulin,bmi,predigree_function,age,target
0,2,84,0,0,0,0.0,0.304,21,0
1,9,112,82,24,0,28.2,1.282,50,1
2,1,139,46,19,83,28.7,0.654,22,0
3,0,161,50,0,0,21.9,0.254,65,0
4,6,134,80,37,370,46.2,0.238,46,1
...,...,...,...,...,...,...,...,...,...
609,5,139,64,35,140,28.6,0.411,26,0
610,1,96,122,0,0,22.4,0.207,27,0
611,10,101,86,37,0,45.6,1.136,38,1
612,0,141,0,0,0,42.4,0.205,29,1


In [11]:
ordi = OrdinalEncoder(categories=[["L", "M", "H"]])
minmax = MinMaxScaler()
onehot = OneHotEncoder()

In [12]:
#all_col = num_col.copy()
#all_col.append("type")

In [13]:
ct = ColumnTransformer([#("ord", ordi, ["type"]),
                        ("minmax", minmax, num_col)], remainder="passthrough")

In [14]:
#new_ct = ColumnTransformer([("minmax", minmax, ["type")], remainder="passthrough")

In [15]:
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('minmax', MinMaxScaler(),
                                 ['pregnancy_occurence',
                                  'glucose_concentration', 'blood_pressure',
                                  'triceps_foldness', 'serum_insulin', 'bmi',
                                  'predigree_function', 'age'])])

In [16]:
train_use = ct.transform(train)
new_features = list(ct.get_feature_names_out())
for i in range(len(new_features)):
    new_features[i] = new_features[i].split("__")[1]
train_df = pd.DataFrame(train_use, columns= new_features)
#train_df["type"] = minmax.fit_transform(np.array(train_df["type"]).reshape(-1,1))
train_df = train_df.clean_names()

In [17]:
test_use = ct.transform(test)
#new_features = list(ct.get_feature_names_out())
test_df = pd.DataFrame(test_use, columns=new_features)
#test_df["type"] = minmax.fit_transform(np.array(test_df["type"]).reshape(-1,1))

In [18]:
test_df = test_df.clean_names()

In [19]:
train_df.columns

Index(['pregnancy_occurence', 'glucose_concentration', 'blood_pressure',
       'triceps_foldness', 'serum_insulin', 'bmi', 'predigree_function', 'age',
       'target'],
      dtype='object')

In [20]:
train_df

,pregnancy_occurence,glucose_concentration,blood_pressure,triceps_foldness,serum_insulin,bmi,predigree_function,age,target
0,0.117647,0.422111,0.000000,0.000000,0.000000,0.000000,0.096499,0.000000,0.0
1,0.529412,0.562814,0.672131,0.380952,0.000000,0.420268,0.514091,0.483333,1.0
2,0.058824,0.698492,0.377049,0.301587,0.098109,0.427720,0.245944,0.016667,0.0
3,0.000000,0.809045,0.409836,0.000000,0.000000,0.326379,0.075149,0.733333,0.0
4,0.352941,0.673367,0.655738,0.587302,0.437352,0.688525,0.068318,0.416667,1.0
...,...,...,...,...,...,...,...,...,...
609,0.294118,0.698492,0.524590,0.555556,0.165485,0.426230,0.142186,0.083333,0.0
610,0.058824,0.482412,1.000000,0.000000,0.000000,0.333830,0.055081,0.100000,0.0
611,0.588235,0.507538,0.704918,0.587302,0.000000,0.679583,0.451751,0.283333,1.0
612,0.000000,0.708543,0.000000,0.000000,0.000000,0.631893,0.054227,0.133333,1.0


In [21]:
train_df.to_csv("../data/train_fe.csv", index=False)
test_df.to_csv("../data/test_fe.csv", index=False)